## STEP 0 - install dependencies

In [1]:
# source is https://www.youtube.com/watch?v=GH3lrOsU3AU
# and notebook is https://github.com/DataTalksClub/llm-zoomcamp/blob/main/0a-agents/notebook.ipynb
# Follow along this tutorial: https://github.com/alexeygrigorev/rag-agents-workshop

# STEP 0 - installing packages we need here and in the VS code terminal:

# Run in VS code terminal:
# pip install --upgrade pip # no reminders after!
# pip install tqdm notebook==7.1.2 openai elasticsearch==8.13.0 pandas scikit-learn ipywidgets
# jupyter notebook # to run jupyter engine locally

In [2]:
%pip install minsearch -q

Note: you may need to restart the kernel to use updated packages.


## STEP 1 - download evaluation data from Github and build in-memory search index with minsearch

In [24]:
# download our FAQ dataset
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

documents[-1] # to see a format of downloaded FAQ dataset

{'text': 'Problem description\nInfrastructure created in AWS with CD-Deploy Action needs to be destroyed\nSolution description\nFrom local:\nterraform init -backend-config="key=mlops-zoomcamp-prod.tfstate" --reconfigure\nterraform destroy --var-file vars/prod.tfvars\nAdded by Erick Calderin',
 'section': 'Module 6: Best practices',
 'question': 'How to destroy infrastructure created via GitHub Actions',
 'course': 'mlops-zoomcamp'}

In [4]:
# build index - takes a few seconds with minsearch

from minsearch import AppendableIndex

index = AppendableIndex(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [5]:
# create a search function with weights: boost = {'question': 3.0, 'section': 0.5}

def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5,
        output_ids=True
    )

    return results

# number of results and course is hard coded here  - filter_dict={'course': 'data-engineering-zoomcamp'}

In [6]:
question = 'Can I still join the course?'

In [7]:
# test if our search function works 

search(question)

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp',
  '_id': 2},
 {'text': "No, you can only get a certificate if you finish the course with a “live” cohort. We don't award certificates for the self-paced mode. The reason is you need to peer-review capstone(s) after submitting a project. You can only peer-review projects at the time the course is running.",
  'section': 'General course-related questions',
  'question': 'Certificate - Can I follow the course in a self-paced mode and get a certificate?',
  'course': 'data-engineering-zoomcamp',
  '_id': 11},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the cou

## STEP 2 - build prompt

In [8]:
# build a prompt

prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

<QUESTION>
{question}
</QUESTION>

<CONTEXT>
{context}
</CONTEXT>
""".strip()

def build_prompt(query, search_results):
    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt



In [10]:
# test how our prompt build works by calling search(question)

prompt = build_prompt(question, search(question))
print(prompt) # better formatted for readability

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

<QUESTION>
Can I still join the course?
</QUESTION>

<CONTEXT>
section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Certificate - Can I follow the course in a self-paced mode and get a certificate?
answer: No, you can only get a certificate if you finish the course with a “live” cohort. We don't award certificates for the self-paced mode. The reason is you need to peer-review capstone(s) after submitting a project. You can only peer-review projects at the time the course is running.

section: General

## STEP 3 - Connect to OpenAI

In [11]:
search_results = search(question)
# just repeating step 1 - keyword search 

In [12]:
prompt = build_prompt(question, search_results)
# repeating step 2 - building prompt on top of search results 

In [19]:
# connecting to LLM, entering our API KEY
import os
from getpass import getpass
from openai import OpenAI

if not (openai_api_key := os.getenv("OPENAI_API_KEY")):
    openai_api_key = getpass("🔑 Enter your OpenAI API key: ")
os.environ["OPENAI_API_KEY"] = openai_api_key


client = OpenAI()



In [16]:
# function to send our prompt to llm 

def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content



In [17]:
# test if our llm can answer something meaningful based on prompt provided

answer = llm(prompt)
print(answer)

Yes, you can still join the course even after the start date. You are eligible to submit the homeworks, but be mindful that there are deadlines for turning in the final projects, so it's advisable not to wait until the last minute.


In [18]:
print(prompt) # this is what we have sent to llm

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

<QUESTION>
Can I still join the course?
</QUESTION>

<CONTEXT>
section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Certificate - Can I follow the course in a self-paced mode and get a certificate?
answer: No, you can only get a certificate if you finish the course with a “live” cohort. We don't award certificates for the self-paced mode. The reason is you need to peer-review capstone(s) after submitting a project. You can only peer-review projects at the time the course is running.

section: General

## STEP 4 - assemble our RAG pipeline

In [20]:
# this is our RAG pipeline

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer



In [21]:
# test by irrelevant question - something which is NOT in FAQ

rag("How do I patch KDE under FreeBSD?")

"I'm sorry, but there is no information available in the provided context regarding how to patch KDE under FreeBSD. Please refer to other resources for guidance on this topic."

In [22]:
rag("What Shakespeare said about peeling a carrot?") 

'The context does not provide any information about what Shakespeare said regarding peeling a carrot. Therefore, I cannot answer that question based on the provided facts.'

## STEP 5 - "Agentic" RAG

In [26]:
# essentially we only modify our prompt

prompt_template = """
You're a course teaching assistant.

You're given a QUESTION from a course student and that you need to answer with your own knowledge and provided CONTEXT.
At the beginning the context is EMPTY.

<QUESTION>
{question}
</QUESTION>

<CONTEXT> 
{context}
</CONTEXT>

If CONTEXT is EMPTY, you can use our FAQ database.
In this case, use the following output template:

{{
"action": "SEARCH",
"reasoning": "<add your reasoning here>"
}}

If you can answer the QUESTION using CONTEXT, use this template:

{{
"action": "ANSWER",
"answer": "<your answer>",
"source": "CONTEXT"
}}

If the context doesn't contain the answer, use your own knowledge to answer the question

{{
"action": "ANSWER",
"answer": "<your answer>",
"source": "OWN_KNOWLEDGE"
}}
""".strip()